In this notebook, we will try to focus in the US dynamic to be able to determine the virus mobility as well as it's R factor depending on different local factors. The use of this notebook is diagnosis on the effectivness of certain measures on the R factor as well as. 

In [1]:
#Los clasicos
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # para leer datos
import sklearn.manifold
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from sklearn.metrics import f1_score
#Ejercicio 16
import sklearn.metrics
import sklearn.tree
#Ejercicio 24
from sklearn.linear_model import LogisticRegression
import sklearn.neural_network
#Ejercicio 20, clusters umap
import sklearn.cluster

#Puede ser interesante el ejercicio 8 para fitear usando bayes, 7 con MCMH .
#No se me puede olvidar la de precondiciones del cdc

One of the biggest problems is to organize our data. The Uncover library has a TON of data. Let's try to make out some cateogries. 

In [49]:
folderset = [folder for folder in glob.glob("Uncover/" + "**/", recursive=True)]

mobility = [folderset[10], folderset[13], folderset[2]]
ressources = [folderset[1], folderset[4], folderset[15], folderset[5]]
healthcareData = [folderset[7],folderset[11]]
covidData = [folderset[3],folderset[6],folderset[8],folderset[14]]
OtherFacilities = [folderset[16]]

csv_Mobility = []
csv_ressources = []
csv_health = []
csv_covid = []
csv_otherFacilities = []

In [50]:
#Mobility
print("\n Mobility \n")
num_carpetas = len(mobility)
for i in range(num_carpetas):
    folder_here = mobility[i] + '*'
    for name in glob.glob(folder_here):
        print(name)
        csv_Mobility.append(name)
#ressources
print("\n Ressources \n")
num_carpetas = len(ressources)
for i in range(num_carpetas):
    folder_here = ressources[i] + '*'
    for name in glob.glob(folder_here):
        print(name)
        csv_ressources.append(name)
#Healthcare
print("\n Healthcare Data \n")
num_carpetas = len(healthcareData)
for i in range(num_carpetas):
    folder_here = healthcareData[i] + '*'
    for name in glob.glob(folder_here):
        print(name)
        csv_health.append(name)
#Covid 
print("\n Covid \n")
num_carpetas = len(covidData)
for i in range(num_carpetas):
    folder_here = covidData[i] + '*'
    for name in glob.glob(folder_here):
        print(name)
        csv_covid.append(name)
#OtherFacilities
print("\n OtherFacilities \n")
num_carpetas = len(OtherFacilities)
for i in range(num_carpetas):
    folder_here = OtherFacilities[i] + '*'
    for name in glob.glob(folder_here):
        print(name)
        csv_otherFacilities.append(name)
                
#We realize that the the school file should be in OtherFacilities

csv_otherFacilities.append(csv_covid.pop(2))

print(csv_otherFacilities)


 Mobility 

Uncover/geotab/border-wait-times-at-us-canada-border.csv
Uncover/geotab/airport-traffic-analysis.csv
Uncover/google_mobility/us-mobility.csv
Uncover/un_world_food_programme/world-travel-restrictions.csv

 Ressources 

Uncover/hifld/us-ports-of-entry.csv
Uncover/hifld/nursing-homes.csv
Uncover/hifld/hospitals.csv
Uncover/hifld/local-emergency-operations-centers-eoc.csv
Uncover/hifld/public-health-departments.csv
Uncover/hifld/urgent-care-facilities.csv
Uncover/hifld/aircraft-landing-facilities.csv
Uncover/esri_covid-19/definitive-healthcare-usa-hospital-beds.csv
Uncover/harvard_global_health_institute/hospital-capacity-by-state-40-population-contracted.csv
Uncover/harvard_global_health_institute/hospital-capacity-by-state-60-population-contracted.csv
Uncover/harvard_global_health_institute/hospital-capacity-by-state-20-population-contracted.csv
Uncover/county_health_rankings/us-county-health-rankings-2020.csv

 Healthcare Data 

Uncover/oecd/health-care-utilization.csv
Unco

Now it is possible to attack one problem at the time. Let's start by developing visualization of the mobility data.  

In [ ]:
data = []
Y = []
largo = len(folderset)
for i in range(largo):
    fruta = folderset[i]
    imagenes = glob.glob("{}/*.jpg".format(fruta))
    print(str(i) +' Fruta')

In [39]:
print(csv_restaurants)

['Uncover/OpenTable/restaurant-performance.csv']


In [9]:
folderset

['Uncover/',
 'Uncover/hifld/',
 'Uncover/un_world_food_programme/',
 'Uncover/HDE/',
 'Uncover/esri_covid-19/',
 'Uncover/county_health_rankings/',
 'Uncover/world_bank/',
 'Uncover/oecd/',
 'Uncover/covid_tracking_project/',
 'Uncover/USAFacts/',
 'Uncover/geotab/',
 'Uncover/us_cdc/',
 'Uncover/ihme/',
 'Uncover/google_mobility/',
 'Uncover/New_York_Times/',
 'Uncover/harvard_global_health_institute/',
 'Uncover/OpenTable/']